# Connected Four - Projeto de Inteligência Artificial

Realizado pelos alunos:
- Rita Moreira (202303885);
- Pedro Gilvaia (202306975);
- Gonçalo Correia (202208527).

## 1. Introdução

O objetivo deste projeto é implementar o algoritmo Monte Carlo Tree Search (MCTS) e uma árvore de decisão para cada dataset dado (iris.csv e o dataset gerado usando o MCTS), usando o algoritmo ID3. Usando esses procedimentos, será então criado um programa capaz de jogar o Quatro em Linha (Connected Four) contra um humano.
Este notebook documenta todo o processo de implementação do jogo, incluindo: 
- explicações das decisões tomadas em cada etapa;
- detalhes da geração de dados com MCTS;
- Treino da árvore de decisão com ID3;
- Integração dos algoritmos no jogo;
- Discussão de resultados.

## 2. Descrição do Problema

O Quatro em Linha é um jogo de estratégia de dois jogadores. São usadas 42 peças, 21 para cada jogador, e um tabuleiro vertical com 7 colunas e 6 filas. A cada jogada, o jogador atual "deixa cair" a sua peça numa coluna à sua escolha, desde que não esteja cheia. Esta cai na linha disponível. Um jogador ganha quando 4 das suas peças formarem uma linha horizontal, vertical ou diagonal consecutiva.

### 2.1. Implementação do Jogo

Abaixo segue-se a implementação das funcionalidades do jogo (connected_four.py). Mais tarde, será apresentada a implementação do jogo em si (game.py), que utiliza todos os códigos trabalhados no projeto (connected_four.py, mcts.py e decision_tree_builder.py).

#### connected_four.py

Importações necessárias:

In [ ]:
from copy import deepcopy
import math

A classe GameMeta funciona como um container de constantes e parâmetros globais que descrevem as regras básicas e convenções do jogo Connect Four na sua implementação. 

In [ ]:
class GameMeta:
    # dicionário que mapeia nomes de jogador para valores usados no jogo
    PLAYERS = {'none': 0, 'one': 1, 'two': 2} 
    # dicionário que mapeia resultados possíveis de uma partida
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    # constante que representa "infinito" em pontuações ou limites
    INF = float('inf')
    # definição do tamanho do tabuleiro (ROWS x COLS)
    ROWS = 6
    COLS = 7

A classe MCTSMeta define o coeficiente de exploração (c). Tem como papel balancear o teste de ramos menos visitados para descobrir se possuem bons resultados (exploit) e a escolha do ramo com maior valor médio $\frac{Q}{N}$. Um c grande amplia a exploração de nós (explore) e um c pequeno prioriza o valor $\frac{Q}{N}$ (exploit).

In [ ]:
class MCTSMeta:
    EXPLORATION = 1.4

A classe ConnectState representa o estado de um jogo. Possui atributos como:
- board: matriz 6x7 com valores 0 (vazio), 1(jogador 1) e 2 (jogador 2);
- to_play: identificador do jogador que jogará a seguir;
- height: lista de índices da próxima linha livre em cada coluna
- last_played: par [linha, coluna] da última jogada.

In [ ]:
class ConnectState:
    def __init__(self):
        '''
        construtor de classe ConnectState
        inicializa o tabuleiro
        o jogador atual
        a altura de cada coluna
        e a última jogada feita
        '''
        # Inicializa o tabuleiro com zeros (vazio)
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        # Inicializa o jogador atual como 'one' (1)
        self.to_play = GameMeta.PLAYERS['one']
        # Inicializa a altura de cada coluna como o número de linhas menos 1 (posição inicial)
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        # Inicializa a lista de ações jogadas como nula
        self.last_played = []

    def get_board(self):
        '''
        retorna uma cópia do tabuleiro atual
        '''
        return deepcopy(self.board)

    def move(self, col):
        '''
        Executa um movimento na coluna especificada
        e atualiza o estado do jogo.
        Se a coluna estiver cheia, levanta um erro (ValueError).
        Parâmetros: col (int) - coluna onde o jogador quer jogar
        '''
        # Verifica se a coluna está cheia
        if self.height[col] < 0:
            raise ValueError(f"Coluna {col} já está cheia!")  # Ou apenas retorna False ou trata o erro de forma apropriada
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

    def get_legal_moves(self):
        '''
        retorna lista de colunas onde ainda é possível jogar (não cheias)
        a lista contém os índices das colunas disponíveis
        '''
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

    def check_win(self):
        '''
        Verifica se o jogador atual venceu.
        Se o jogador atual venceu, retorna o código do jogador (1 ou 2).
        Se não houver vitória, retorna 0.
        '''
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0


    def check_win_from(self, row, col):
        """
        Verifica se há quatro em linha a partir da posição (row, col)
        em todas as direções (horizontal, vertical, diagonal e anti-diagonal).
        Se houver quatro em linha, retorna True, caso contrário, retorna False.
        Parâmetros: row e col da última jogada
        """
        player = self.board[row][col]
        consecutive = 1
        # Check horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # Check vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # Check diagonal
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # Check anti-diagonal
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False

    def game_over(self):
        '''
        verifica se o jogo acabou
        se o jogo acabou, retorna True
        se não acabou, retorna False
        '''
        return self.check_win() or len(self.get_legal_moves()) == 0

    def get_outcome(self):
        '''
        retorna o código de resultado da partida (outcomes)
        1 ou 2 se for vitória
        3 se for empate
        '''
        if len(self.get_legal_moves()) == 0 and self.check_win() == 0:
            return GameMeta.OUTCOMES['draw']

        return GameMeta.OUTCOMES['one'] if self.check_win() == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']

    # exibe o tabuleiro no terminal 
    # mapeia 1->X, 2->O, 0->' '
    def print(self):
        print('=============================')

        for row in range(GameMeta.ROWS):
            for col in range(GameMeta.COLS):
                print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
            print('|')

        print('=============================')


## 3. Metodologia e sua implementação

### 3.1. Monte Carlo Tree Search (MCTS)

O MCTS é um algoritmo de busca adversarial que leva em consideração as alterações de estado causadas por um adversário nas etapas subsequentes. A sua implementação utiliza o critério Upper Confidence Bound (UCT) para avaliar cada ramo de uma árvore, de fórmula:
$$
UCT = \frac{Q}{N} + c\sqrt{\frac{In{N_{parent}}}{N}}
$$
O MCTS tem quatro fases:
- Seleção: Começa na raiz e, em cada nó, escolhe o filho que maximiza o UCT até atingir um nó ainda não totalmente expandido;
- Expansão:  A partir do nó selecionado, gera um dos estados-filho ainda não representados na árvore, adicionando-o como novo nó;
- Simulação: A partir desse nó recém-criado, executa um jogo até ao fim (playout), usando uma política aleatória ou heurística, obtendo um resultado (vitória/derrota);
- Retropropagação (Backpropagation): Propaga o resultado da simulação de volta à raiz, atualizando em cada nó visitado. Nesses nós, a contagem de visitas aumenta 1 valor e contagem de vitórias aumenta o valor do resultado (0 se perdeu, 1 se ganhou).


#### mcts.py

Importações necessárias:

In [ ]:
import random
import time
import math
from copy import deepcopy

from connected_four import ConnectState, GameMeta, MCTSMeta

A classe Node representa um nó de uma árvore de pesquisa Monte Carlo. Tem como atributos:
- move: jogada que levou ao nó atual;
- parent: nó pai da árvore (a razi tem parent=None);
- N: número de visitas;
- Q: Soma das recompensas (vitórias) obtidas deste nó;
- children: mapeia jogadas para nós-filho;
- outcome: estado de resultado no nó (0=nenhum, 1=jogador1, 2=jogador2).

In [ ]:
class Node:
    '''
    Inicialização de um nó
    '''
    def __init__(self, move, parent):
        self.move = move
        self.parent = parent
        self.N = 0
        self.Q = 0
        self.children = {}
        self.outcome = GameMeta.PLAYERS['none']

    def add_children(self, children: dict) -> None:
        '''
        adiciona nós filho a partir de um dicionário
        Parâmetros: children (dict) - dicionário de nós a inserir como filhos
        '''
        for child in children:
            self.children[child.move] = child
    
    def get_exploration(self) -> float:
        '''
        Calcula dinamicamente o coeficiente de exploração c para UCT.
        Baseia-se no número de visitas do nó pai.
        Retorna o valor de c ajustado.
        C diminui conforma a raiz quadrada do número de visitas do nó pai 
        aumenta.
        Leva a uma exploração mais intensa no início 
        e uma exploração mais leve no final.

        '''
        c0 = MCTSMeta.EXPLORATION #1.4
        if self.parent is not None:
            root_visits = self.parent.N 
            alpha = 0.2
            return c0 / (1 +  alpha * math.log(1 + root_visits))
        

    def value(self, explore: float = MCTSMeta.EXPLORATION):
        '''
        Calcula o valor UCT do nó.
        Retorna o valor UCT para a seleção.
        '''
        if self.N == 0:
            return GameMeta.INF # forçar exploração de nós não visitados
        else: #dynamic c
            c_now = self.get_exploration()
            return self.Q / self.N + c_now * math.sqrt(math.log(self.parent.N) / self.N)


A classe MCTS implementa o algoritmo para o Connected Four. Tem como atributos:
- root_state: estado de jogo na raiz;
- root: nó raiz da árvore de busca;
- run_time: tempo gasto na última pesquisa (segundos de CPU);
- num_rollouts: número de simulações executadas na última pesquisa.

In [ ]:
class MCTS:
    def __init__(self, state=ConnectState()):
        '''
        Inicializa o MCTS com um estado inicial, inicializado pela classe ConnectedState.
        '''
        self.root_state = deepcopy(state)
        self.root = Node(None, None)
        self.run_time = 0
        self.node_count = 0
        self.num_rollouts = 0

    def select_node(self) -> tuple:
        '''
        Fase seleção:
        Seleciona o nó a ser expandido com base na política UCT
        Até que um nó não visitado seja encontrado ou o jogo acabe.
        Retorna o nó selecionado e o estado do jogo.
        '''
        node = self.root
        state = deepcopy(self.root_state)

        # Se o nó não tiver filhos, expande-o
        while len(node.children) != 0:
            # escolher o filho com maior valor UCT
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            max_nodes = [n for n in children if n.value() == max_value]

            node = random.choice(max_nodes)
            state.move(node.move)

            if node.N == 0:
                return node, state
        # se o nó não for terminal, expande-o
        if self.expand(node, state):
            node = random.choice(list(node.children.values()))
            state.move(node.move)

        return node, state

    def expand(self, parent: Node, state: ConnectState) -> bool:
        '''
        Fase de expansão:
        Cria filhos para o nó pai com base nas jogadas legais disponíveis.
        Retorna False se o estado for terminal (sem expansão possível).
        Parâmetros: 
        parent (Node) - nó pai a ser expandido
        state (ConnectState) - estado atual do jogo
        '''
        if state.game_over():
            return False

        children = [Node(move, parent) for move in state.get_legal_moves()]
        parent.add_children(children)

        return True

    def roll_out(self, state: ConnectState) -> int:
        '''
        Fase de simulação:
        Joga aleatoriamente até o fim do jogo.
        Retorna o resultado do jogo (1, 2 ou 3).
        '''
        while not state.game_over():
            moves = state.get_legal_moves()
            player = state.to_play
            opp = 3 - player
            # Vitória imediata
            for m in moves:
                s2 = deepcopy(state)
                s2.move(m)
                if s2.game_over() and s2.get_outcome() == player:
                    state.move(m)
                    break
            else:
                # Bloqueio de ameaças adversárias
                blocked = False
                for m in moves:
                    s2 = deepcopy(state)
                    s2.move(m)
                    for m2 in s2.get_legal_moves():
                        s3 = deepcopy(s2)
                        s3.move(m2)
                        if s3.game_over() and s3.get_outcome() == opp:
                            state.move(m)
                            blocked = True
                            break
                    if blocked:
                        break
                if not blocked:
                    state.move(random.choice(moves))

        return state.get_outcome()

    def back_propagate(self, node: Node, turn: int, outcome: int) -> None:
        '''
        Fase de retropropagação/backpropagation:
        Atualiza os valores de N e Q dos nós visitados
        Parâmetros:
        node (Node) - nó onde terminou a simulação
        turn (int) - jogador que fez a última jogada
        outcome (int) - resultado do jogo (1, 2 ou 3)
        '''
        #recompensa relativa ao jogador 'turn'
        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.Q += reward
            node = node.parent
            if outcome == GameMeta.OUTCOMES['draw']:
                reward = 0
            else:
                reward = 1 - reward

    def search(self, time_limit: int):
        '''
        Executa simulações de MCTS por um tempo limitado.
        Parâmetros: time_limit (int) - tempo máximo de CPU para simulações
        '''
        start_time = time.process_time()

        num_rollouts = 0
        while time.process_time() - start_time < time_limit:
            node, state = self.select_node()
            outcome = self.roll_out(state)
            self.back_propagate(node, state.to_play, outcome)
            num_rollouts += 1

        run_time = time.process_time() - start_time
        self.run_time = run_time
        self.num_rollouts = num_rollouts

    def best_move(self):
        '''
        Retorna a melhor jogada com base no número de visitas (N) a partir da raiz
        Retorna a coluna escolhida com o melhor movimento ou -1 se o jogo já acabou.
        '''
        if self.root_state.game_over():
            return -1

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        best_child = random.choice(max_nodes)

        return best_child.move

    def move(self, move):
        '''
        Atualiza a raiz para refletir um movimento externo
        Parâmetros: move (int) - coluna onde o jogador jogou
        '''
        if move in self.root.children:
            self.root_state.move(move)
            self.root = self.root.children[move]
            return

        self.root_state.move(move)
        self.root = Node(None, None)

    def statistics(self) -> tuple:
        '''
        Retorna as estatísticas da última pesquisa
        como o número de simulações e o tempo gasto.
        '''
        return self.num_rollouts, self.run_time


### 3.2. Árvores de Decisão

Para o segundo método, foi utilizado um dataset (iris.csv) para testar ("warm-up") o algoritmo ID3. É um dataset mais pequeno, de contexto diferente, que tem como tarefa gerar/treinar uma árvore de decisão que, dadas quatro características (comprimento e largura da pétala, comprimento e largura da sépala), determine a que classe cada planta pertence (Iris setosa, Iris virginica ou Iris versicolor).
Após esse teste, o objetivo é gerar uma árvore de decisão associada à implementação do algoritmo MCTS. É gerado um conjunto de pares ($state_{i}, move_{i}$), em que $state_{i}$ é o estado corrente do jogo e $move_{i}$ é o próximo movimento sugerido pelo algoritmo. Com esse dataset (mcts_dataset.csv), é treinada uma árvore de decisão usando o algoritmo ID3 (Iterative Dichotomiser 3), de forma a que, dado um estado de jogo, a árvore escolha o próximo movimento.
Abaixo serão apresentadas os ficheiros decision_tree_builder.py, que implementa o algoritmo ID3, e iris_test.py, que aplica esse algoritmo no seu dataset.


#### decision_tree_builder.py

Importações necessárias:

In [ ]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

A classe Node representa um nó numa árvore de decisão ID3. Tem como atributos:
- feature: nome ou índice da feature usada para dividir neste nó (None se folha);
- children: mapeamento do valor de feature para nós-filho;
- label: Rótulo da classe se este nó for folha (None caso contrário).

In [ ]:
class Node:
    def __init__(self, feature=None, children=None, label=None):
        self.feature = feature
        self.children = children if children is not None else {}
        self.label = label

In [ ]:
def entropy(y):
    '''
    Calcula a entropia de Shannon de um vetor de rótulos y.
    Parâmetros: 
    y (pd.Series) - Série de rótulos/classes
    Retorna: Entropia H(y) = -sum(p_i * log2(p_i))
    '''
    counts = Counter(y)
    probabilities = [count / len(y) for count in counts.values()]
    return -sum(p * np.log2(p) for p in probabilities if p > 0)

def information_gain(X, y, feature):
    '''
    Calcula o ganho de informação ao dividir X e y pela feature especificada.
    Parâmetros:
    X (pd.DataFrame) - DataFrame de atributos
    y (pd.Series) - Série de rótulos/classes
    feature - nome da coluna de X a avaliar
    Retorna: IG = H(y) - sum(|subset|/|y| * H(subset_y))
    '''
    values = X[feature].unique()
    weighted_entropy = 0

    for v in values:
        subset_y = y[X[feature] == v]
        weighted_entropy += (len(subset_y) / len(y)) * entropy(subset_y)

    return entropy(y) - weighted_entropy

def id3(X, y, features, depth=0, max_depth=15):
    '''
    Constrói uma árvore de decisão usando o algoritmo ID3.
    Parâmetros:
    X (pd.DataFrame) - DataFrame de atributos (conjunto de treino)
    y (pd.Series) - Série de rótulos/classes correspondente
    features (list) - lista de colunas ainda disponíveis para divisão
    depth (int) - profundidade atual da árvore
    max_depth (int) - profundidade máxima permitida
    Retorna: nó raiz da árvore de decisão
    '''
    # critérios de paragem: pureza, sem features ou profundidade máxima
    if len(set(y)) == 1 or len(features) == 0 or depth == max_depth:
        return Node(label=y.iloc[0])
    
    
    if len(features) == 0:
        most_common_label = y.mode()[0]
        return Node(label=most_common_label)
    #escolha de melhor feature
    gains = [information_gain(X, y, f) for f in features]
    best_feature = features[np.argmax(gains)]

    node = Node(feature=best_feature)
    feature_values = X[best_feature].unique()
    #recursão para cada valor da feature escolhida
    for value in feature_values:
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]

        if len(subset_y) == 0:
            # nenhum exemplo com esse valor
            most_common_label = y.mode()[0]
            child = Node(label=most_common_label)
        else:
            #continua a recursão sem a feature escolhida
            remaining_features = [f for f in features if f != best_feature]
            child = id3(subset_X, subset_y, remaining_features, depth+1, max_depth)

        node.children[value] = child

    return node
def predict(tree, sample):
    '''
    Prediz a classe de um exemplo usando a árvore de decisão.
    Parâmetros:
    tree (Node) - nó raiz da árvore de decisão
    sample - exemplo a classificar
    Retorna: classe prevista ou None se não houver previsão
    '''
    while tree.label is None:
        value = sample.get(tree.feature)
        if value in tree.children:
            tree = tree.children[value]
        else:
            # Valor nunca visto — retorna o valor de maior ocorrência entre os filhos
            # ou uma jogada aleatória segura
            if tree.children:
                return majority_vote(tree)
            else:
                return None
    return tree.label

def majority_vote(node):
    '''
    Para ramos nunca vistos durante a previsão,
    faz votação majoritária nos descendentes
    Parâmetros:
    node (Node) - nó atual da árvore de decisão
    Retorna: classe mais comum entre os filhos ou None se não houver filhos
    '''
    labels = []
    def collect_labels(subnode):
        if subnode.label is not None:
            labels.append(subnode.label)
        else:
            for child in subnode.children.values():
                collect_labels(child)
    collect_labels(node)
    if labels:
        return Counter(labels).most_common(1)[0][0]
    else:
        return None


In [ ]:
# Carrega dados discretizados
data = pd.read_csv("mcts_dataset.csv")
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
features = X.columns.tolist()

# Treina a árvore de decisão
tree = id3(X, y, features)

# Testa a árvore de decisão
sample = X.iloc[0].to_dict()  # primeira linha como dict
predicted_label = predict(tree, sample)
actual_label = y.iloc[0]

#print("Predicted:", predicted_label)
#print("Actual:   ", actual_label)
correct = 0
for i in range(len(X)):
    sample = X.iloc[i].to_dict()
    prediction = predict(tree, sample)
    if prediction == y.iloc[i]:
        correct += 1

accuracy = correct / len(X)
#print(f"Accuracy on training data: {accuracy:.2%}")

# Exporta a árvore de decisão
with open("decision_tree.pkl", "wb") as f:
    pickle.dump(tree, f)

#### iris_test.py

Importações necessárias:

In [ ]:
import pandas as pd
import pickle
from decision_tree_builder import id3, predict, Node

In [ ]:
# Carregamento e discretização dos dados do dataset iris.csv

df = pd.read_csv('iris.csv')

def discretize(col, n_bins=3):
    '''
    Discretiza uma coluna num número fixo de bins.
    Parâmetros:
    col (pd.Series) - coluna a discretizar
    n_bins (int) - número de bins (default: 3)
    Retorna: Coluna com os valores discretizados
    '''
    return pd.cut(col, bins=n_bins, labels=[f'bin{i}' for i in range(n_bins)])

# Discretiza todas as colunas, exceto a última (rótulo)
for col in df.columns[:-1]:
    df[col] = discretize(df[col])

# Preparação de atributos e rótulos
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
features = X.columns.tolist()

# Divisão em treino e teste (70% treino, 30% teste)
df_shuffled = df.sample(frac=1).reset_index(drop=True)
split_idx = int(len(df_shuffled) * 0.7)
train = df_shuffled.iloc[:split_idx].reset_index(drop=True)
test = df_shuffled.iloc[split_idx:].reset_index(drop=True)

X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]

# treino da árvore de decisão
tree = id3(X_train, y_train, features)

# Serialização da árvore de decisão com pickle
with open('iris_tree.pkl', 'wb') as f:
    pickle.dump(tree, f)
print("Decision tree serialized to iris_tree.pkl")

# Função para imprimir a árvore de decisão
def print_tree(node: Node, depth=0):
    '''
    Exibe a estrutura da árvore de decisão de forma recursiva
    Parâmetros:
    node (Node) - nó atual da árvore de decisão
    depth (int) - profundidade atual da árvore
    '''
    indent = '  ' * depth
    if node.label is not None:
        print(f"{indent}→ Label: {node.label}")
    else:
        feat_name = features[node.feature] if isinstance(node.feature, int) else node.feature
        print(f"{indent}Feature: {feat_name}")
        for value, child in node.children.items():
            print(f"{indent}  If == {value}:")
            print_tree(child, depth+2)

# Exibe a estrutura da árvore de decisão gerada
print("\nDecision Tree Structure:")
print_tree(tree)

# Avalia a acurácia no conjunto de teste
correct = 0
for idx, row in X_test.iterrows():
    sample = row.to_dict()
    pred = predict(tree, sample)
    true_label = y_test.loc[idx]
    if pred == true_label:
        correct += 1
accuracy = correct / len(X_test)
print(f"\nAccuracy: {accuracy:.2%}")


## 4. Integração no Jogo

Usando os códigos acima apresentados, é então possível recriar o jogo Quatro em Linha. Em baixo está o código final, que importa todos os anteriores.

#### game.py

Importações necessárias:

In [ ]:
from connected_four import ConnectState, GameMeta  # Representa o estado do jogo
from mcts import MCTS                   # Implementa o algoritmo MCTS
import pickle
from decision_tree_builder import predict  # Importa a função de previsão da árvore de decisão

In [ ]:
# Exceções específicas para controlo de fluxo
# permitem reiniciar ou sair do jogo de forma controlada

class RestartGameException(Exception):
    pass

class QuitGameException(Exception):
    pass

In [ ]:
# carregamento da árvore de decisão (ficheiro .pkl)
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)

In [ ]:
#Lê inteiros com tratamento para "restart" e "quit"
def safe_int_input(prompt):
    while True:
        try:
            value = input(prompt)
            if value.lower() == 'restart':
                raise RestartGameException
            if value.lower() == 'quit':
                raise QuitGameException
            return int(value)
        except ValueError:
            print("Entrada inválida. Por favor insira um número inteiro.")
        except EOFError:
            raise

# Lê uma jogada válida (coluna de 0 a 6)
def safe_move_input(state, prompt="Escolha uma coluna (0-6), 'restart' ou 'quit': "):
    while True:
        try:
            user_input = input(prompt)
            if user_input.lower() == 'restart':
                raise RestartGameException
            if user_input.lower() == 'quit':
                raise QuitGameException
            move = int(user_input)
            if move in state.get_legal_moves():
                return move
            print("Movimento inválido. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Insira um número de 0 a 6.")
        except EOFError:
            raise
# Função para converter o estado do jogo em features
#Transforma o estado do tabuleiro em um dicionário de atributos para alimentar a árvore de decisão
def state_to_features(state):
    # Assumindo que o estado tem uma representação interna `board`
    # e que a árvore foi treinada com features nomeadas como pos_0, pos_1, ..., pos_41
    features = {}
    for i in range(6):  # 6 linhas
        for j in range(7):  # 7 colunas
            index = i * 7 + j
            features[f"s{index}"] = state.board[i][j]

    return features



In [ ]:
# Funções de jogo com retorno de resultado

def play_player_vs_player(starting_player):
    state = ConnectState()
    state.to_play = starting_player
    while not state.game_over():
        state.print()
        print(f"Vez de {'X' if state.to_play==1 else 'O'}")
        move = safe_move_input(state)
        state.move(move)
    state.print()
    return state.get_outcome()

def play_player_vs_tree(player_piece, first, tree):
    state = ConnectState()
    tree_piece = 2 if player_piece == 1 else 1

    if first == 'tree':
        state.to_play = tree_piece
        state.print()
        features = state_to_features(state)
        mv = predict(tree, features)
        print(f"Árvore ({'X' if tree_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)
    else:
        state.to_play = player_piece

    while not state.game_over():
        state.print()
        if state.to_play == player_piece:
            mv = safe_move_input(state)
        else:
            print("Árvore a pensar...")
            features = state_to_features(state)
            mv = predict(tree, features)
            print(f"Árvore ({'X' if tree_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)

    state.print()
    return state.get_outcome()


def play_player_vs_ai(player_piece, first):
    state = ConnectState()
    ai_piece = 2 if player_piece == 1 else 1
    mcts = MCTS(state)
    if first == 'ai':
        state.to_play = ai_piece
        state.print()
        print("MCTS a começar...")
        mcts.search(10)
        mv = mcts.best_move()
        print(f"MCTS ({'X' if ai_piece==1 else 'O'}) escolheu: {mv}")
        state.move(mv)
        mcts.move(mv)
    else:
        state.to_play = player_piece
    while not state.game_over():
        state.print()
        if state.to_play == player_piece:
            mv = safe_move_input(state)
            state.move(mv)
            mcts.move(mv)
        else:
            print("MCTS a pensar...")
            mcts.search(5)
            mv = mcts.best_move()
            print(f"MCTS ({'X' if ai_piece==1 else 'O'}) escolheu: {mv}")
            state.move(mv)
            mcts.move(mv)
    state.print()
    return state.get_outcome()

def play_ai_vs_tree(starting_player, tree):
    state = ConnectState()
    mcts = MCTS(state)
    ai_piece = starting_player
    tree_piece = 2 if ai_piece == 1 else 1

    while not state.game_over():
        state.print()
        if state.to_play == ai_piece:
            print("MCTS a pensar...")
            mcts.search(5)
            mv = mcts.best_move()
            print(f"MCTS escolheu: {mv}")
        else:
            print("Árvore a pensar...")
            features = state_to_features(state)
            mv = predict(tree, features)
            print(f"Árvore escolheu: {mv}")
        state.move(mv)
        mcts.move(mv)

    state.print()
    return state.get_outcome()


def play_ai_vs_ai(starting_player):
    state = ConnectState()
    state.to_play = starting_player
    mcts1, mcts2 = MCTS(state), MCTS(state)
    while not state.game_over():
        state.print()
        # Indica qual IA está a jogar
        current_ai = 'X' if state.to_play == 1 else 'O'
        print(f"MCTS {current_ai} a pensar...")
        # Execução MCTS conforme a peça
        if state.to_play == 1:
            mcts1.search(5)
            mv = mcts1.best_move()
        else:
            mcts2.search(5)
            mv = mcts2.best_move()
        # Mostra a jogada escolhida
        print(f"MCTS {current_ai} escolheu a coluna: {mv}")
        # Aplica a jogada e atualiza ambas as árvores
        state.move(mv)
        mcts1.move(mv)
        mcts2.move(mv)
    state.print()
    return state.get_outcome()

# Menu pós-jogo: Play Again, Change Modes, Quit
def post_game_menu():
    print("O que deseja fazer a seguir?")
    print("1: Jogar novamente")
    print("2: Mudar modo")
    print("3: Sair")
    choice = safe_int_input("Escolha: ")
    if choice == 1:
        return 'again'
    if choice == 2:
        raise RestartGameException
    return 'quit'


In [ ]:
# Bloco principal

def main():
    x_wins = o_wins = draws = 0
    try:
        while True:
            print("1: Jogador vs Jogador\n2: Jogador vs MCTS\n3: Jogador vs Árvore\n4: MCTS vs Árvore")
            mode = safe_int_input("Modo: ")
            # Definições iniciais do modo
            if mode == 1:
                print("Quem começa? \n1: X  \n2: O")
                first = 1 if safe_int_input("Escolha: ")==1 else 2
                while True:
                    outcome = play_player_vs_player(first)
                    if outcome == 1: x_wins += 1
                    elif outcome == 2: o_wins += 1
                    else: draws += 1
                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException
            elif mode == 2:
                print("Quem começa? \n1: Jogador  \n2: MCTS")
                first = 'player' if safe_int_input("Escolha: ") == 1 else 'ai'
                print("Que peça prefere? \n1: X  \n2: O")
                player_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                mcts_piece = 3 - player_piece

                while True:
                    outcome = play_player_vs_ai(player_piece, first)
                    # se outcome == player_piece → jogador humano ganhou
                    if outcome == player_piece:
                        if player_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == mcts_piece:
                        # MCTS ganhou
                        if mcts_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException

            elif mode == 3:
                print("Quem começa? \n1: Jogador  \n2: Árvore")
                first = 'player' if safe_int_input("Escolha: ") == 1 else 'tree'
                print("Que peça prefere? \n1: X  \n2: O")
                player_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                tree_piece = 3 - player_piece

                while True:
                    outcome = play_player_vs_tree(player_piece, first, decision_tree)
                    if outcome == player_piece:
                        # Jogador humano ganhou
                        if player_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == tree_piece:
                        # Árvore ganhou
                        if tree_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException

                    
            elif mode == 4:
                print("Quem começa? \n1: MCTS  \n2: Árvore")
                mcts_piece = 1 if safe_int_input("Escolha: ") == 1 else 2
                tree_piece = 3 - mcts_piece

                while True:
                    outcome = play_ai_vs_tree(mcts_piece, decision_tree)
                    if outcome == mcts_piece:
                        # MCTS ganhou
                        if mcts_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    elif outcome == tree_piece:
                        # Árvore ganhou
                        if tree_piece == 1:
                            x_wins += 1
                        else:
                            o_wins += 1
                    else:
                        draws += 1

                    print(f"Placar: (X) {x_wins} - {o_wins} (O) (Empates: {draws})")
                    action = post_game_menu()
                    if action == 'again':
                        continue
                    elif action == 'quit':
                        raise QuitGameException



            else:
                print("Modo inválido.")
    except RestartGameException:
        print("Mudando modo...")
        main()
    except QuitGameException:
        print("Programa encerrado.")
    except EOFError:
        print("Entrada finalizada.")

if __name__ == "__main__":
    main()
# Este código implementa um jogo de Conecta 4 com várias opções de modo de jogo.

## 4. Resultados

## 5. Discussão dos Resultados (pode se juntar ao cap 4)

## 6. Conclusão 

## 7. Referências

Qi Wang’s Blog! (2022). Connect 4 with Monte Carlo Tree Search. [online] Available at: https://www.harrycodes.com/blog/monte-carlo-tree-search.
‌